In [2]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

driver = webdriver.Chrome('chromedriver.exe')
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(10)

email = '아이디 입력'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '비밀번호'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

def insta_search(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url 

url = insta_search('전기차충전')
driver.get(url)
time.sleep(50)

def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(5)

select_first(driver)

import re, unicodedata

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    try:
        content = soup.select('div.C4VMK > span')[0].text  
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    
    tags = re.findall(r'#[^s#,\\]+',content)
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except:
        like = 0
    
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    data = [content, date, like, place, tags]
    return data 

# get_content(driver)

def move_next(driver): # 다음 게시글 조회
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

results = [ ] 

target = 500
for i in tqdm(range(target)):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(2)
        move_next(driver)

# print(results[:2])

import pandas as pd 

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']
results_df.to_csv('./insta_charge.csv',index=False, encoding = 'utf-8-sig')

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [27:02<00:00,  3.25s/it]


In [3]:
df = pd.read_csv('insta_charge.csv')

In [6]:
df

,content,date,like,place,tags
0,동네 전기차 충전시설 이 정도면 주말 충전하기 딱 좋은.#너무좋다☺️ #이마트죽전점...,2020-11-29,0.0,이마트 죽전점,"['#너무좋다☺️ ', '#이마트죽전점', '#전기차충전 ', '#테슬라처럼 ', ..."
1,🚘유튜브 홍시car🚘 아이오닉5 롱레인지 프레스티지 시승기 업로드 완료! #yout...,2021-04-23,0.0,NaN,"['#youtube ', '#naverTV ', '#hong', '#hyundai ..."
2,-전기차는 맞는데... 😅.-#키즈카 #kidscar #키즈카튜닝 #kidscart...,2019-09-02,0.0,NaN,"['#키즈카 ', '#kid', '#키즈카튜닝 ', '#kid', '#전동차 ', ..."
3,⠀메르세데스-벤츠만의 프리미엄 충전 솔루션을 경험해보세요.Electric Intel...,2019-11-14,0.0,NaN,[]
4,#유료광고날씨 좋으니까 르노 조에랑 드라이브 하다 떠난 즉흥여행:D 지리산-대구 9...,2020-10-03,0.0,NaN,['#유료광고날씨 좋으니까 르노 조에랑 드라이브 하다 떠난 즉흥여행:D 지리산-대구...
...,...,...,...,...,...
489,#에이바우트 #에이바우트이벤트 #에이바우트초이스이벤트 #닌텐도 #닌텐도주세요 #주말...,2021-05-22,0.0,에이바우트커피 첨단점,"['#에이바우트 ', '#에이바우트이벤트 ', '#에이바우트초이스이벤트 ', '#닌..."
490,#마이브#초소형전기차#트위지#쎄보c#전기차#출퇴근차#업무용차#배달대행 #배민커넥트#...,2021-05-22,0.0,NaN,"['#마이브', '#초소형전기차', '#트위지', '#쎄보c', '#전기차', '#..."
491,경기도 김포시 단독주택 🏠1층 피아노 레슨실🎹 2층 주거공간피아니스트 및 음대 교수...,2021-05-22,0.0,NaN,[]
492,전기차 보급량이 점점 늘어나군요^^ #친환경 #전기차 #테슬라 #전기차충전,2021-05-21,0.0,NaN,"['#친환경 ', '#전기차 ', '#테슬라 ', '#전기차충전']"


In [5]:
df.value_counts('place')

place
Gumi          6
제주시 중앙로       5
Yangyang      4
제주도시재생지원센터    3
Daejeon       3
             ..
고속도로          1
경주            1
강릉            1
가평군 설악면       1
효안과           1
Length: 129, dtype: int64

In [12]:
df['place'].dropna()

0                 이마트 죽전점
6                    Gumi
16     Daegu, South Korea
20                미사강변신도시
21             장안평중고차매매센터
              ...        
483                법무법인강남
484          Seoul, Korea
485                  Gumi
489           에이바우트커피 첨단점
493             한남동 유엔빌리지
Name: place, Length: 161, dtype: object